In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

%matplotlib inline

Load CSV Files

In [2]:
covid_df = pd.read_csv('data/COVID_CountByZipPerDate 03292021.csv')
vaccine_df = pd.read_csv('data/vaccine_with_protocol.csv')

In [33]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   zip         8832 non-null   object
 1   month_year  8832 non-null   object
 2   cases       8832 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 207.1+ KB


In [3]:
covid_df.Zip = covid_df.Zip.astype('str')
vaccine_df.zipcode = vaccine_df.zipcode.astype('str')
vaccine_df.count_given = vaccine_df.count_given.astype('float')
vaccine_df.dose_nbr = vaccine_df.dose_nbr.astype('Int64')

Rename Columns

In [4]:
vaccine_df.columns = ['date', 'zip', 'v_count', 'dose_nbr', 'protocol']
covid_df.columns = ['date', 'zip', 'cases']

In [5]:
vaccine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6694 entries, 0 to 6693
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      6694 non-null   object 
 1   zip       6694 non-null   object 
 2   v_count   6686 non-null   float64
 3   dose_nbr  6686 non-null   Int64  
 4   protocol  6694 non-null   object 
dtypes: Int64(1), float64(1), object(3)
memory usage: 268.1+ KB


Convert Date Time

In [6]:
covid_df.date = pd.to_datetime(covid_df.date)
vaccine_df.date = pd.to_datetime(vaccine_df.date)

group by zipcode to get total cases per zipcode

In [7]:
#What does this mean?*******************

In [8]:
covid_df_g = covid_df.groupby(['zip'], as_index=False)['cases'].sum()
vaccine_df_g = vaccine_df.groupby(['zip'], as_index=False)['v_count'].sum()

Create Month_Year Column

In [9]:
vaccine_df2 = vaccine_df
covid_df2 = covid_df

In [10]:
vaccine_df2['month'] = vaccine_df2['date'].dt.month
vaccine_df2['year'] = vaccine_df2['date'].dt.year

In [11]:
covid_df2['month'] = covid_df2['date'].dt.month
covid_df2['year'] = covid_df2['date'].dt.year

In [12]:
vaccine_df2['month_year'] = vaccine_df2['month'].map(str) + '-' + vaccine_df2['year'].map(str)
covid_df2['month_year'] = covid_df2['month'].map(str) + '-' + covid_df2['year'].map(str)

In [13]:
vaccine_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6694 entries, 0 to 6693
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        6694 non-null   datetime64[ns]
 1   zip         6694 non-null   object        
 2   v_count     6686 non-null   float64       
 3   dose_nbr    6686 non-null   Int64         
 4   protocol    6694 non-null   object        
 5   month       6694 non-null   int64         
 6   year        6694 non-null   int64         
 7   month_year  6694 non-null   object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 425.0+ KB


Is he renaming the columns with the double brackets?

In [14]:
covid_df = covid_df[['zip', 'month_year', 'cases']]
vaccine_df = vaccine_df[['month_year','zip','v_count','dose_nbr','protocol']]

In [15]:
#why can't these be called by v_my_g.info()? ***********

In [16]:
v_my_g = vaccine_df2.groupby(['zip'])
c_my_g = covid_df2.groupby(['zip'])

In [17]:
ethnicity_df = pd.read_excel("data/latino_excel.xlsx", skiprows = 1, header=[0,1], sheet_name=0)
ethnicity_df = ethnicity_df[ethnicity_df.columns.drop(list(ethnicity_df.filter(regex='Error')))]
ethnicity_df.columns = ethnicity_df.columns.droplevel(1)

In [18]:
ethnicity_df.head()

,Unnamed: 0_level_0,37010,37012,37013,37014,37015,37016,37020,37022,37025,...,38482,38483,38487,38547,38548,38552,38560,38563,38567,38569
0,Total:,4966,2185,97939,2835,17665,950,5984,6583,5698,...,1635,5910,1044,1939,931,157,1405,3410,685,69
1,Not Hispanic or Latino:,4803,2134,81131,2783,16768,944,5824,6286,5698,...,1574,5748,1022,1912,931,157,1405,3410,660,69
2,White alone,4502,2050,40080,2682,15842,939,5137,5968,5526,...,1525,5608,838,1884,930,157,1307,3202,615,69
3,Black or African American alone,245,73,33280,18,304,0,222,173,24,...,0,31,113,0,0,0,0,140,0,0
4,American Indian and Alaska Native alone,2,0,95,0,147,0,99,32,0,...,0,0,0,28,1,0,2,0,0,0


#what does this mean in english? ****************

In [22]:
ethnicity_df_clean = ethnicity_df.set_index('Unnamed: 0_level_0').T.reset_index().rename({'index' : 'zip'}, axis=1)

In [23]:
ethnicity_df_clean.head()

Unnamed: 0_level_0,zip,Total:,Not Hispanic or Latino:,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races:,...,Hispanic or Latino:,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races:,Two races including Some other race,"Two races excluding Some other race, and three or more races"
0,37010,4966,4803,4502,245,2,27,0,0,27,...,163,90,0,0,0,20,0,53,0,53
1,37012,2185,2134,2050,73,0,3,0,0,8,...,51,47,0,0,0,0,0,4,0,4
2,37013,97939,81131,40080,33280,95,4250,0,374,3052,...,16808,13242,446,48,0,0,2464,608,357,251
3,37014,2835,2783,2682,18,0,47,0,0,36,...,52,40,0,0,0,0,0,12,12,0
4,37015,17665,16768,15842,304,147,107,0,5,363,...,897,635,0,2,0,0,221,39,37,2


In [24]:
pop_by_zip = ethnicity_df_clean[['zip','Total:']]
pop_by_zip.rename(columns={'Total:':'total'})

Unnamed: 0_level_0,zip,total
0,37010,4966
1,37012,2185
2,37013,97939
3,37014,2835
4,37015,17665
...,...,...
113,38552,157
114,38560,1405
115,38563,3410
116,38567,685


In [25]:
Load geospatial zips

SyntaxError: invalid syntax (<ipython-input-25-79bfa5255501>, line 1)

In [26]:
zipcodes = gpd.read_file('data/zipcodes.geojson')
print(zipcodes.crs)
# zipcodes

epsg:4326


In [27]:
vac_zip = zipcodes.merge(vaccine_df_g, how='inner', left_on='zip', right_on='zip')

In [28]:
covid_zip = zipcodes.merge(covid_df_g, how="inner", left_on='zip', right_on='zip')

In [29]:
vac_zip = vac_zip[['zip', 'shape_stlength', 'shape_starea', 'geometry','v_count']]

In [30]:
vaccine_df_g.head()

,zip,v_count
0,37013.0,27167.0
1,37027.0,506.0
2,37069.0,89.0
3,37072.0,13108.0
4,37076.0,16855.0


In [36]:
zipcodes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   zip             56 non-null     object  
 1   objectid        56 non-null     object  
 2   po_name         56 non-null     object  
 3   shape_stlength  56 non-null     object  
 4   shape_starea    56 non-null     object  
 5   geometry        56 non-null     geometry
dtypes: geometry(1), object(5)
memory usage: 2.8+ KB


In [38]:
covid_df_g.head()

,zip,cases
0,37013.0,13354
1,37027.0,231
2,37069.0,16
3,37072.0,2984
4,37076.0,4279


In [95]:
covid_df_g.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   zip     32 non-null     object
 1   cases   32 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 768.0+ bytes


In [97]:
covid_df_g.zip = covid_df_g.zip.astype(float)

In [99]:
covid_df_g.head()

,zip,cases
0,37013.0,13354
1,37027.0,231
2,37069.0,16
3,37072.0,2984
4,37076.0,4279


In [101]:
covid_df_g.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   zip     31 non-null     float64
 1   cases   32 non-null     int64  
dtypes: float64(1), int64(1)
memory usage: 768.0 bytes
